### NASA satellite data download API via AppEEARS
Author: Yahampath A Marambe; email:  yahampath.marambe@utah.edu  

Code adpoted from the example code from LPDAAC

In [45]:
# import packages
import requests as r
import getpass, pprint, time, os, cgi, json
import geopandas as gpd


### USER inputs here
Give username, password and task name when the code prompts for inputs  
following cell is dedicated to change product, bands, date range, and coordinates

In [46]:
# mention the product here
prods = ['L08.002'] # look for this product id in https://appeears.earthdatacloud.nasa.gov/products 

# layers for download
layers = [(prods[0], 'SR_B2'),
          #(prods[0], 'SR_B3'),
          #(prods[0], 'SR_B4'),
          #(prods[0], 'SR_B5'),
          #(prods[0], 'SR_B6'),
          #(prods[0], 'SR_B7')
          ]
# select the task type
tsk_type = [
            'point',
            #'area'
            ]  

# day ranges
start  = '05-01-2023' # MM-DD-YYYY
end = '08-01-2023' # MM-DD-YYYY

# GeoJSON coordinate object to submit task for different regions
coordinates = [{
        "id": "0",
        "longitude": "-119.03368536943579",
        "latitude": "36.93119616355591",
        "category": "SN"
        }, {
        "id": "1", 
        "longitude": "-105.74880383393888",
        "latitude": "40.14930109570027",
        "category": "ARP"
        }]

### USER inpu ends

In [47]:
# set the output directory
inDir = '.'
# api as a variable
api = 'https://appeears.earthdatacloud.nasa.gov/api/' 
 # Input NASA Earthdata Login Username and password
user = getpass.getpass(prompt = 'Enter NASA Earthdata Login Username: ')     
password = getpass.getpass(prompt = 'Enter NASA Earthdata Login Password: ')
# take a token for the task
token_response = r.post('{}login'.format(api), auth=(user, password)).json()
# delete the user name and password for safety
del user, password 
print(f'The token response is {token_response}')


The token response is {'message': "The server could not verify that you are authorized to access the URL requested. You either supplied the wrong credentials (e.g. a bad password), or your browser doesn't understand how to supply the credentials required."}


In [44]:
# print number of aviable products
product_response = r.get('{}product'.format(api)).json()
print('AρρEEARS currently supports {} products.'.format(len(product_response)))
# Create all product names and indexes
products = {p['ProductAndVersion']: p for p in product_response}
# print all products if need
print(f'These are the aviable products')
#pprint.pprint(products)
# take the product you want with indexing and print all information
prd = products['L08.002'] # look for this product id in https://appeears.earthdatacloud.nasa.gov/products 
# print it with all info
print('These are selected product informations')
pprint.pprint(prd)
# give the product of interrest
#prods = ['L08.002'] # this is already metioned above
# see all the response / subdata /bands for this product
lst_response = r.get('{}product/{}'.format(api, prods[0])).json()
print('Availbe bands for the product')
pprint.pprint(list(lst_response.keys()))
# and just try printing a band discription
print(lst_response['SR_B2']['Description'])
# mention layers: keep comment if mentioned above
#layers = [(prods[0], 'SR_B2'),
#          (prods[0], 'SR_B3'),
#          (prods[0], 'SR_B4')]
# print layers for downloading
print('*******************')
print('Fowllowing layers will be downloaded')
pprint.pprint(layers)
# create a list of dictonery contain all layers product combinations
prodLayer = []
for l in layers:
    prodLayer.append({
            "layer": l[1],
            "product": l[0]
          })
# layer product combinations
print('Layer product combinations')
pprint.pprint(prodLayer)

# point request submission
# save login token 
token = token_response['token'] 
# create a header to submit a request
head = {'Authorization': 'Bearer {}'.format(token)}
# give a task name to current task
task_name = input('Enter a Task Name: ')
#task_type = ['point','area']  
task_type = tsk_type 
startDate = start   
endDate = end       
recurring = False          
#yearRange = [2000,2016]    # If recurring = True, set yearRange, change start/end date to MM-DD



AρρEEARS currently supports 155 products.
These are the aviable products
These are selected product informations
{'Available': True,
 'DOI': '10.5066/P960F8OC',
 'Deleted': False,
 'Description': 'Landsat Collection 2 ARD Surface Reflectance - Landsat 8',
 'DocLink': 'https://www.usgs.gov/landsat-missions/landsat-collection-2-us-analysis-ready-data',
 'Info': {'extension': 'tif',
          'platform': 'LANDSAT_8',
          'provider_id': 'landsat_prod'},
 'Platform': 'Landsat ARD',
 'Product': 'L08',
 'ProductAndVersion': 'L08.002',
 'RasterType': 'Tile',
 'Resolution': '30m',
 'Source': 'USGS',
 'TemporalExtentEnd': 'Present',
 'TemporalExtentStart': '2013-03-18',
 'TemporalGranularity': '16 day',
 'Version': '002'}
Availbe bands for the product
['QA_LINEAGE',
 'QA_PIXEL',
 'QA_RADSAT',
 'SR_B1',
 'SR_B2',
 'SR_B3',
 'SR_B4',
 'SR_B5',
 'SR_B6',
 'SR_B7',
 'SR_QA_AEROSOL']
Band 2 Surface Reflectance
*******************
Fowllowing layers will be downloaded
[('L08.002', 'SR_B2'),
 ('L0

### Task building and submitting task

In [49]:
task = {
    'task_type': task_type[0],
    'task_name': task_name,
    'params': {
         'dates': [
         {
             'startDate': startDate,
             'endDate': endDate
         }],
         'layers': prodLayer,
         'coordinates': coordinates
    }
}
pprint.pprint(task)

# send task to api
task_response = r.post('{}task'.format(api), json=task, headers=head).json()
print(task_response)
# limit to 2 recent entries
params = {'limit': 2, 'pretty': True} 
# Query task service setting params & header
tasks_response = r.get('{}task'.format(api),params = params, headers=head).json() 
print(tasks_response)

# task id for sumission
task_id = task_response['task_id']                    
status_response = r.get('{}status/{}'.format(api, task_id), headers=head).json() 
print(status_response)

# Ping API until request is complete ; live update of status
starttime = time.time()
while r.get('{}task/{}'.format(api, task_id), headers=head).json()['status'] != 'done':
    print(r.get('{}task/{}'.format(api, task_id), headers=head).json()['status'])
    time.sleep(20.0 - ((time.time() - starttime) % 20.0))
print(r.get('{}task/{}'.format(api, task_id), headers=head).json()['status'])

{'params': {'coordinates': [{'category': 'SN',
                             'id': '0',
                             'latitude': '36.93119616355591',
                             'longitude': '-119.03368536943579'},
                            {'category': 'ARP',
                             'id': '1',
                             'latitude': '40.14930109570027',
                             'longitude': '-105.74880383393888'}],
            'dates': [{'endDate': '08-01-2023', 'startDate': '05-01-2023'}],
            'layers': [{'layer': 'SR_B2', 'product': 'L08.002'},
                       {'layer': 'SR_B3', 'product': 'L08.002'},
                       {'layer': 'SR_B4', 'product': 'L08.002'},
                       {'layer': 'SR_B5', 'product': 'L08.002'},
                       {'layer': 'SR_B6', 'product': 'L08.002'},
                       {'layer': 'SR_B7', 'product': 'L08.002'}]},
 'task_name': 'task1',
 'task_type': 'point'}
{'task_id': 'eb707ed2-8f96-4d62-b608-9df358a56dc3', '

In [50]:
# set destination 
destDir = os.path.join(inDir, task_name)                
if not os.path.exists(destDir):os.makedirs(destDir)     # Create the output directory

bundle = r.get('{}bundle/{}'.format(api,task_id), headers=head).json()  # Call API and return bundle contents for the task_id as json
print(bundle)

files = {}                                                       # Create empty dictionary
for f in bundle['files']: files[f['file_id']] = f['file_name']   # Fill dictionary with file_id as keys and file_name as values
print(files)


# downloading files
for f in files:
    dl = r.get('{}bundle/{}/{}'.format(api, task_id, f), headers=head, stream=True, allow_redirects = "TRUE")  # Get a stream to the bundle file
    if files[f].endswith('.tif'):
        filename = files[f].split('/')[1]
    else:
        filename = files[f]
    filepath = os.path.join(destDir, filename)                                            # Create output file path
    with open(filepath, 'wb') as f:                                                       # Write file to dest dir
        for data in dl.iter_content(chunk_size=8192): f.write(data) 
print('Downloaded files can be found at: {}'.format(destDir))


{'files': [{'sha256': 'fe6b09d4fc05bae861beb5d7741f8f3c017a7c062ddb7bcc22a54fdc76084f0a', 'file_id': 'bd659263-55f6-4b9d-a3ed-c911fddb829c', 'file_name': 'task1-L08-002-results.csv', 'file_size': 12346, 'file_type': 'csv', 's3_url': 's3://appeears-output/eb707ed2-8f96-4d62-b608-9df358a56dc3/task1-L08-002-results.csv'}, {'sha256': '7ad0326c77ba20a30ce8ecd782acb64f3543ef3135a834ad6688665453a3792b', 'file_id': 'b66800b1-bc39-4ac8-91aa-44aaddec4023', 'file_name': 'task1-granule-list.txt', 'file_size': 17160, 'file_type': 'txt', 's3_url': 's3://appeears-output/eb707ed2-8f96-4d62-b608-9df358a56dc3/task1-granule-list.txt'}, {'sha256': '6ba14c9a0c549e452906d310b076300a0609ba0dfa5d94cff4e3c1415e4d4184', 'file_id': '75b881b5-10d4-46ab-9c1a-9ae4f21cdd23', 'file_name': 'task1-request.json', 'file_size': 1070, 'file_type': 'json', 's3_url': 's3://appeears-output/eb707ed2-8f96-4d62-b608-9df358a56dc3/task1-request.json'}, {'sha256': '90d53604e81bc68db572f32f2c8b3de689df86b9fbdfd49601af9bea36a5d040', 

Note:
Read the granule list file, you can use those tif file paths with request packge to download them to the local computer. For, testing just copy paste the path in the web browse and image will start to download.

In [ ]:
### Read the